In [2]:
import gzip
import json

def messages_parse(data):
    data_with_no_binary = str(data).split("'")[1:-1][0]
    data_list =[i for i in data_with_no_binary.split('\\n') if i != '']
    array = [json.loads(D) for D in data_list]
    return array 

def messages_read(file="../data/coinbase_BTC-USD_20_10_06_000000-010000.json.gz"):
    file_object = gzip.open(file, "r")
    data = file_object.read()
    messages_data = messages_parse(data)
    return messages_data 

def snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_00.json"):
    snaphsot_data = json.loads(open(file,'r').read())
    return snaphsot_data

messages_data = messages_read()
initial_clob = snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_00.json")
final_clob = snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_15.json")

In [4]:
print (messages_data[0])
print (initial_clob.keys())
print (final_clob.keys())

{'type': 'done', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:00:00.021717Z', 'sequence': 16712803594, 'order_id': '0e612005-8b3c-4b34-a2db-eadc20afc95e', 'reason': 'canceled', 'price': '10802.46', 'remaining_size': '0.25'}
dict_keys(['bids', 'asks', 'sequence'])
dict_keys(['bids', 'asks', 'sequence'])
